In [1]:
from pandas import DataFrame, read_csv, concat
from csv import reader
from numpy import mean, array, zeros, errstate, seterr, isfinite
from collections import defaultdict
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr
from os import path
from gensim.models import KeyedVectors, Word2Vec
from pymorphy2 import MorphAnalyzer
from itertools import product
import string

translator = str.maketrans('', '', string.punctuation)
morph = MorphAnalyzer()
seterr(all='raise')

Using Theano backend.


{'divide': 'ignore', 'invalid': 'ignore', 'over': 'ignore', 'under': 'ignore'}

In [12]:
# df_en = DataFrame.from_csv('data/eye-tracking/eye_tracking_data_en_raw.csv').reset_index(drop=True)

In [13]:
# df_ru = DataFrame.from_csv('data/eye-tracking/eye_tracking_data_ru.csv')

In [51]:
# geco_raw = DataFrame.from_csv('data/eye-tracking/geco_raw.csv')

Load raw messy corpus proposed by the Laboratory of Neurolinguistics and transform it to a decent dataset

In [52]:
# with open(path.join('data', 'data.csv'), mode='r') as infile:
#     corpus_bytes = reader(infile)
#     corpus = [i for i in corpus_bytes]

# corpus = [i[0].split('\t') if len(i) == 1 else ''.join(i).split('\t') for i in corpus]
# df = DataFrame(corpus[1:], columns=corpus[0])

Amount of unique words in a corpus

In [53]:
# len(df['word.id'].unique())

Aggregate corpus data by unqiue words

In [54]:
# geco_raw.WORD = geco_raw.WORD.apply(lambda x: x.translate(translator).lower().strip())
# geco_raw = geco_raw.reset_index()

In [55]:
# geco_raw = geco_raw.astype('str').groupby('WORD').agg({
#                             'TRIAL_TOTAL_READING_TIME': ', '.join,
#                             'WORD_ID_WITHIN_TRIAL': ', '.join,
#                             'WORD_AVERAGE_FIX_PUPIL_SIZE': ', '.join,
#                             'WORD_FIXATION_COUNT': ', '.join,
#                             'WORD_FIXATION_%': ', '.join,
#                             'WORD_RUN_COUNT': ', '.join,
#                             'WORD_FIRST_RUN_START_TIME': ', '.join,
#                             'WORD_FIRST_RUN_END_TIME': ', '.join,
#                             'WORD_FIRST_RUN_FIXATION_COUNT': ', '.join,
#                             'WORD_FIRST_RUN_FIXATION_%': ', '.join,
#                             'WORD_GAZE_DURATION': ', '.join,
#                             'WORD_SECOND_RUN_START_TIME': ', '.join,
#                             'WORD_SECOND_RUN_END_TIME': ', '.join,
#                             'WORD_SECOND_RUN_FIXATION_COUNT': ', '.join,
#                             'WORD_SECOND_RUN_FIXATION_%': ', '.join,
#                             'WORD_THIRD_RUN_START_TIME': ', '.join,
#                             'WORD_THIRD_RUN_END_TIME': ', '.join,
#                             'WORD_THIRD_RUN_FIXATION_COUNT': ', '.join,
#                             'WORD_THIRD_RUN_FIXATION_%': ', '.join,
#                             'WORD_FIRST_FIXATION_DURATION': ', '.join,
#                             'WORD_FIRST_FIXATION_INDEX': ', '.join,
#                             'WORD_FIRST_FIXATION_RUN_INDEX': ', '.join,
#                             'WORD_FIRST_FIXATION_TIME': ', '.join,
#                             'WORD_FIRST_FIXATION_VISITED_WORD_COUNT': ', '.join,
#                             'WORD_FIRST_FIXATION_X': ', '.join,
#                             'WORD_FIRST_FIXATION_Y': ', '.join,
#                             'WORD_FIRST_FIX_PROGRESSIVE': ', '.join,
#                             'WORD_SECOND_FIXATION_DURATION': ', '.join,
#                             'WORD_SECOND_FIXATION_RUN': ', '.join,
#                             'WORD_SECOND_FIXATION_TIME': ', '.join,
#                             'WORD_SECOND_FIXATION_X': ', '.join,
#                             'WORD_SECOND_FIXATION_Y': ', '.join,
#                             'WORD_THIRD_FIXATION_DURATION': ', '.join,
#                             'WORD_THIRD_FIXATION_RUN': ', '.join,
#                             'WORD_THIRD_FIXATION_TIME': ', '.join,
#                             'WORD_THIRD_FIXATION_X': ', '.join,
#                             'WORD_THIRD_FIXATION_Y': ', '.join,
#                             'WORD_LAST_FIXATION_DURATION': ', '.join,
#                             'WORD_LAST_FIXATION_RUN': ', '.join,
#                             'WORD_LAST_FIXATION_TIME': ', '.join,
#                             'WORD_LAST_FIXATION_X': ', '.join,
#                             'WORD_LAST_FIXATION_Y': ', '.join,
#                             'WORD_GO_PAST_TIME': ', '.join,
#                             'WORD_SELECTIVE_GO_PAST_TIME': ', '.join,
#                             'WORD_TOTAL_READING_TIME': ', '.join,
#                             'WORD_TOTAL_READING_TIME_%': ', '.join,
#                             'WORD_SPILLOVER': ', '.join,
#                             'WORD_SKIP': ', '.join,
#                         }).reset_index()

In [16]:
# df_en = df_en.astype('str').groupby('Word_Cleaned').agg({
#                             'IA_DWELL_TIME': ', '.join,
#                             'IA_FIRST_FIXATION_DURATION': ', '.join,
#                             'IA_FIRST_FIXATION_VISITED_IA_COUNT': ', '.join,
#                             'IA_FIRST_FIXATION_X': ', '.join,
#                             'IA_FIRST_FIXATION_Y': ', '.join,
#                             'IA_FIRST_FIX_PROGRESSIVE': ', '.join,
#                             'IA_FIRST_FIXATION_TIME': ', '.join,
#                             'IA_FIRST_RUN_DWELL_TIME': ', '.join,
#                             'IA_FIRST_RUN_START_TIME': ', '.join,
#                             'IA_FIRST_RUN_END_TIME': ', '.join,
#                             'IA_FIRST_FIXATION_TIME': ', '.join,
#                             'IA_FIRST_RUN_FIXATION_COUNT': ', '.join,
#                             'IA_DWELL_TIME': ', '.join,
#                             'IA_FIXATION_COUNT': ', '.join,
#                             'IA_RUN_COUNT': ', '.join,
#                             'IA_SKIP': ', '.join,
#                             'IA_REGRESSION_IN': ', '.join,
#                             'IA_REGRESSION_IN_COUNT': ', '.join,
#                             'IA_REGRESSION_OUT': ', '.join,
#                             'IA_REGRESSION_OUT_COUNT': ', '.join,
#                             'IA_REGRESSION_OUT_FULL': ', '.join,
#                             'IA_REGRESSION_OUT_FULL_COUNT': ', '.join,
#                             'IA_REGRESSION_PATH_DURATION': ', '.join,
#                             'IA_REGRESSION_OUT_FULL_COUNT': ', '.join,
#                             'IA_REGRESSION_PATH_DURATION': ', '.join,
#                             'IA_FIRST_SACCADE_AMPLITUDE': ', '.join,
#                             'IA_FIRST_SACCADE_ANGLE': ', '.join,
#                             'IA_FIRST_SACCADE_END_TIME': ', '.join,
#                             'IA_FIRST_SACCADE_START_TIME': ', '.join
#                         }).reset_index()

In [17]:
# df = df.groupby('Lemma').agg({
#                             'average.accuracy': ', '.join,
#                             'IA_DWELL_TIME': ', '.join,
#                             'IA_FIRST_FIXATION_DURATION': ', '.join,
#                             'IA_FIRST_RUN_DWELL_TIME': ', '.join,
#                             'IA_FIRST_RUN_FIXATION_COUNT': ', '.join,
#                             'IA_FIXATION_COUNT': ', '.join,
#                             'IA_LEGAL': ', '.join,
#                             'IA_REGRESSION_IN': ', '.join,
#                             'IA_REGRESSION_OUT_FULL': ', '.join,
#                             'IA_REGRESSION_PATH_DURATION': ', '.join,
#                             'IA_SECOND_RUN_DWELL_TIME': ', '.join,
#                             'ao': ', '.join,
#                             'IA_SELECTIVE_REGRESSION_PATH_DURATION': ', '.join,
#                             'IA_SKIP': ', '.join,
#                             'IA_SPILLOVER': ', '.join,
#                             'landing': ', '.join,
#                             'dir': ', '.join,
#                             'fixated.letter': ', '.join,
#                             'one_fix': ', '.join,
#                             'twoplus_fix': ', '.join,
#                             'logit.acc': ', '.join,
#                         }).reset_index()

Serialize transformed dataset

In [85]:
# df_ru.to_csv('data/eye-tracking/eye_tracking_data_ru.csv')
df_ru = DataFrame.from_csv('data/eye-tracking/eye_tracking_data_ru.csv').fillna(',')

In [3]:
# df_en[7:].reset_index(drop=True).to_csv('data/eye-tracking/eye_tracking_data_en.csv')
df_en = DataFrame.from_csv('data/eye-tracking/eye_tracking_data_en.csv').fillna(',')

In [60]:
# geco_raw.to_csv('data/eye-tracking/geco.csv')
geco = DataFrame.from_csv('data/eye-tracking/geco.csv').fillna(',')

In [61]:
ru_en_dict = DataFrame.from_csv('data/dictionaries/en-ru-dict.csv')

Calculate mean value of aggregated values for each word in the dataset

In [62]:
def calculate_mean(df, na):
    for column in df:
        for i in range(len(df)):
            try:
                values = list(filter(lambda a: a != na, [a.strip() for a in df[column][i].split(',')]))
                if not values:
                    df[column][i] = 0
                    continue
                df[column][i] = mean([float(a) for a in values])
            except ValueError: # if column is a column of words
                continue 
            except FloatingPointError: # if all are NA
                df[column][i] = 0
    return df

In [115]:
df_en = calculate_mean(df_en, 'nan')
df_ru = calculate_mean(df_ru, 'NA')

Loading and processing dataset of human judgements of embeddings

In [64]:
def load_sim_dataset(df, embeddings, verbose=False, rusvectores=False):
    # df = read_csv(name)
    old_len = len(df)
    for i, m in df.iterrows():
        if not rusvectores:
            if not m['word1'] in embeddings or not m['word2'] in embeddings:
                df.drop(i, inplace=True)
        else:
            if not add_pos_tag(m['word1']) in embeddings or not add_pos_tag(m['word2']) in embeddings:
                df.drop(i, inplace=True)
    if verbose:
        print('Percent of dropped = {:2.1f}%, amount of remanining words = {}'.format((old_len - len(df))/old_len*100, len(df)))
    return df.reset_index(drop=True)

In [65]:
def load_en_sim_dataset(df, embeddings, verbose=False, rusvectores=False):
    # df = read_csv(name)
    old_len = len(df)
    for i, m in df.iterrows():
        try:
            if not ru_en_dict[ru_en_dict['ru'] == m['word1']].iloc[0, 0] in embeddings or not ru_en_dict[ru_en_dict['ru'] == m['word2']].iloc[0, 0] in embeddings:
                df.drop(i, inplace=True)
        except IndexError:
            df.drop(i, inplace=True)
    if verbose:
        print('Percent of dropped = {:2.1f}%, amount of remanining words = {}'.format((old_len - len(df))/old_len*100, len(df)))
    return df.reset_index(drop=True)

In [66]:
def load_ru_sim_dataset(df, embeddings, verbose=False, rusvectores=False):
    # df = read_csv(name)
    old_len = len(df)
    for i, m in df.iterrows():
        try:
            if not add_pos_tag(ru_en_dict[ru_en_dict['en'] == m['word1']].iloc[0, 1]) in embeddings or not add_pos_tag(ru_en_dict[ru_en_dict['en'] == m['word2']].iloc[0, 1]) in embeddings:
                df.drop(i, inplace=True)
        except IndexError:
            df.drop(i, inplace=True)
    if verbose:
        print('Percent of dropped = {:2.1f}%, amount of remanining words = {}'.format((old_len - len(df))/old_len*100, len(df)))
    return df.reset_index(drop=True)

In [67]:
def make_sims_dataset(dataset, embeddings, rusvectores=False):
    sims = zeros(shape=len(dataset), dtype='float32')
    for i, m in dataset.iterrows():
        if not rusvectores:
            sims[i] = 1 - cosine(embeddings[m['word1']], embeddings[m['word2']])
        else:
            sims[i] = 1 - cosine(embeddings[add_pos_tag(m['word1'])], embeddings[add_pos_tag(m['word2'])])
    return sims

In [68]:
def make_en_sims_dataset(dataset, embeddings, rusvectores=False):
    sims = []
    for i, m in dataset.iterrows():
        
        try:
            sims.append(1 - cosine(embeddings[ru_en_dict[ru_en_dict['ru'] == m['word1']].iloc[0, 0]], embeddings[ru_en_dict[ru_en_dict['ru'] == m['word2']].iloc[0, 0]]))
        except IndexError:
            continue
    return array(sims)

In [69]:
def make_ru_sims_dataset(dataset, embeddings, rusvectores=False):
    sims = []
    for i, m in dataset.iterrows():
        try:
            sims.append(1 - cosine(embeddings[add_pos_tag(ru_en_dict[ru_en_dict['en'] == m['word1']].iloc[0, 1])], 
                        embeddings[add_pos_tag(ru_en_dict[ru_en_dict['en'] == m['word2']].iloc[0, 1])]))
        except IndexError:
            continue
    return array(sims)

In [70]:
def make_mapped_datasets_en_ru(dataset_en, dataset_ru):
    sims_en = []
    sims_ru = []
    for i, m in dataset_en.iterrows():
        try:
            sims_ru.append(dataset_ru[(dataset_ru['word1'] == morph.parse(ru_en_dict[ru_en_dict['en'] == m['word1']].iloc[0, 1])[0].normal_form) 
                & (dataset_ru['word2'] == morph.parse(ru_en_dict[ru_en_dict['en'] == m['word2']].iloc[0, 1])[0].normal_form)].iloc[0, 2])
            sims_en.append(m['similarity'])
        except IndexError:
            continue
    return array(sims_en), array(sims_ru)

In [71]:
def make_mapped_datasets_ru_en(dataset_ru, dataset_en):
    sims_ru = []
    sims_en = []
    for i, m in dataset_ru.iterrows():
        try:
            sims_en.append(dataset_ru[(dataset_ru['word1'] == ru_en_dict[ru_en_dict['ru'] == m['word1']].iloc[0, 0]) 
                & (dataset_ru['word2'] == ru_en_dict[ru_en_dict['ru'] == m['word2']].iloc[0, 0])].iloc[0, 2])
            sims_ru.append(m['sim'])
        except IndexError:
            continue
    return array(sims_en), array(sims_ru)

In [72]:
def make_word2vec_dataset(dataset, model):
    sims = zeros(shape=len(dataset), dtype='float32')
    for i, m in dataset.iterrows():
        sims[i] = 1 - cosine(model[add_pos_tag(m['word1'])], model[add_pos_tag(m['word2'])])
    return sims

In [73]:
def make_word2vec_eye_dataset(dataset, model, lang='ru'):
    pairs = list(product(dataset, repeat=2))
    sims_w2v = zeros(shape=len(pairs), dtype='float32')
    sims_eye = zeros(shape=len(pairs), dtype='float32')
    for i in enumerate(pairs):
        try:
            if lang == 'en':
                sims_w2v[i[0]] = 1 - cosine(model[i[1][0]], model[i[1][1]])
                sims_eye[i[0]] = 1 - cosine(dataset[i[1][0]], dataset[i[1][1]])
            else: # add pos tags
                sims_w2v[i[0]] = 1 - cosine(model[add_pos_tag(i[1][0])], model[add_pos_tag(i[1][1])])
                sims_eye[i[0]] = 1 - cosine(dataset[i[1][0]], dataset[i[1][1]])
        except KeyError:
            continue
    return sims_w2v, sims_eye

Load Word2Vec-model

In [74]:
rusvectores = KeyedVectors.load_word2vec_format(path.join('..', '..', 'word2vec-models', 'ruwikiruscorpora.bin'), binary=True)

In [75]:
googlenews = KeyedVectors.load_word2vec_format(path.join('..',  '..', 'word2vec-models', 'google-news.bin'), binary=True)

In [76]:
def add_pos_tag(word):
    tag = str(morph.parse(word)[0].tag.POS)
    if tag == 'ADJF':
        tag = 'ADJ'
    elif tag == 'INFN':
        tag = 'VERB'
    if word == 'объем': 
        tag = 'NOUN'
    if word == 'струя':
        tag = 'NOUN'
    if word == 'чай':
        tag = 'NOUN'
    if word == 'два':
        word = 'двадцать'
        tag = 'NUM'
    return '{}_{}'.format(word, tag)

Make a comparison

In [116]:
eye_embeddings_en = {}
eye_embeddings_en_ru = {}

for i, k in df_en.iloc[:,0:len(df_en.columns)].iterrows():
    try:
        eye_embeddings_en_ru[ru_en_dict[ru_en_dict['en'] == k['word']].iloc[0, 1]] = array(k[1:].values)
        eye_embeddings_en[k['word']] = array(k[1:].values)
    except IndexError: # no such word in the en-ru dict
        continue

In [117]:
eye_embeddings_ru = {}
eye_embeddings_ru_en = {}

for i, k in df_ru.iloc[:,0:len(df_ru.columns)].iterrows():
    try:
        eye_embeddings_ru_en[ru_en_dict[ru_en_dict['ru'] == k['word']].iloc[0, 0]] = array(k[1:].values)
        eye_embeddings_ru[k['word']] = array(k[1:].values)
    except IndexError: # no such word in the en-ru dict
        continue

In [118]:
rw = concat([read_csv(_[0]) for _ in russian_word_similarity]).reset_index(drop=True)

In [119]:
english_word_similarity = [
                ('../../evaluation/en/word-similarity/simverb-3500.csv', 'SimVerb-3500'),
                ('../../evaluation/en/word-similarity/men.csv', 'MEN'),
                ('../../evaluation/en/word-similarity/rw.csv', 'Rare Word'),
                ('../../evaluation/en/word-similarity/simlex999.csv', 'SimLex999'),
                ('../../evaluation/en/word-similarity/mturk-771.csv', 'MTurk-771'),
                # ('../../evaluation/en/word-similarity/semeva2017.csv', 'SemEval-2017'),
                ('../../evaluation/en/word-similarity/wordsim353-rel.csv', 'WordSim353 Relatedness'),
                ('../../evaluation/en/word-similarity/wordsim353-sim.csv', 'WordSim353 Similarity'),
                ('../../evaluation/en/word-similarity/mturk-287.csv', 'MTurk-287'),
                ('../../evaluation/en/word-similarity/verb-143.csv', 'Verb-143'),
                ('../../evaluation/en/word-similarity/yp-130.csv', 'YP-130'),
                ('../../evaluation/en/word-similarity/rg-65.csv', 'RG-65'),
                ('../../evaluation/en/word-similarity/mc-30.csv', 'MC-30'),
                ]

In [120]:
ew = concat([read_csv(_[0]) for _ in english_word_similarity]).reset_index(drop=True)

In [121]:
df = rw.copy()
dataset = load_sim_dataset(df, eye_embeddings_ru, True)
eye_sims_ru = make_sims_dataset(dataset, eye_embeddings_ru)
print('Correlation with Russian human judgements of Russian gaze vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(eye_sims_ru, dataset.similarity)))
print()

df = rw.copy()
dataset = load_sim_dataset(df, eye_embeddings_en_ru, True)
eye_sims_en = make_sims_dataset(dataset, eye_embeddings_en_ru)
print('Correlation with Russian human judgements of English gaze vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(eye_sims_en, dataset.similarity)))

Percent of dropped = 98.5%, amount of remanining words = 21
Correlation with Russian human judgements of Russian gaze vectors: 0.57 (0.01)

Percent of dropped = 99.6%, amount of remanining words = 6
Correlation with Russian human judgements of English gaze vectors: 0.03 (0.96)


In [122]:
df = ew.copy()
dataset = load_sim_dataset(df, eye_embeddings_en, True)
eye_sims_en = make_sims_dataset(dataset, eye_embeddings_en)
print('Correlation with English human judgements of English gaze vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(eye_sims_en, dataset.similarity)))
print()

df = ew.copy()
dataset = load_sim_dataset(df, eye_embeddings_ru_en, True)
eye_sims_ru = make_sims_dataset(dataset, eye_embeddings_ru_en)
print('Correlation with English human judgements of Russian gaze vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(eye_sims_ru, dataset.similarity)))

Percent of dropped = 98.0%, amount of remanining words = 232
Correlation with English human judgements of English gaze vectors: -0.03 (0.60)

Percent of dropped = 99.7%, amount of remanining words = 37
Correlation with English human judgements of Russian gaze vectors: 0.31 (0.06)


In [123]:
seterr(all='ignore')

{'divide': 'raise', 'invalid': 'raise', 'over': 'raise', 'under': 'raise'}

In [124]:
print('Correlation with Russian gaze vectors of Russian Word2Vec: {:0.2f} ({:0.2f})'.format(*spearmanr(*make_word2vec_eye_dataset(eye_embeddings_ru, rusvectores))))
print('Correlation with English gaze vectors of Russian Word2Vec: {:0.2f} ({:0.2f})'.format(*spearmanr(*make_word2vec_eye_dataset(eye_embeddings_en_ru, rusvectores))))

Correlation with Russian gaze vectors of Russian Word2Vec: 0.06 (0.00)
Correlation with English gaze vectors of Russian Word2Vec: 0.08 (0.00)


In [125]:
print('Correlation with Russian gaze vectors of English Word2Vec: {:0.2f} ({:0.2f})'.format(*spearmanr(*make_word2vec_eye_dataset(eye_embeddings_ru_en, googlenews, 'en'))))
print('Correlation with English gaze vectors of English Word2Vec: {:0.2f} ({:0.2f})'.format(*spearmanr(*make_word2vec_eye_dataset(eye_embeddings_en, googlenews, 'en'))))

Correlation with Russian gaze vectors of English Word2Vec: 0.14 (0.00)
Correlation with English gaze vectors of English Word2Vec: 0.09 (0.00)


In [78]:
df_en = ew.copy()
df_ru = rw.copy()

print('Correlation of English human judgements with Russian human judgements: {:0.2f} ({:0.2f})'.
      format(*spearmanr(*make_mapped_datasets_en_ru(df_en, df_ru))))
print()

Correlation of English human judgements with Russian human judgements: -0.03 (0.54)



/Users/defeater/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [79]:
df_en = ew.copy()
df_ru = rw.copy()

print('Correlation of Russian human judgements with English human judgements: {:0.2f} ({:0.2f})'.
     format(*spearmanr(*make_mapped_datasets_ru_en(df_ru, df_en))))
print()

Correlation of Russian human judgements with English human judgements: nan (nan)



In [80]:
df_ru = rw.copy()

dataset = load_sim_dataset(df_ru, rusvectores, True, True)
dataset.to_csv('temp.csv')
dataset = load_en_sim_dataset(read_csv('temp.csv'), googlenews, True)

w2v_en = make_en_sims_dataset(dataset, googlenews)
print('Correlation with Russian human judgements of English word2vec vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(w2v_en, dataset.similarity)))
print()

w2v_ru = make_sims_dataset(dataset, rusvectores, rusvectores=True)
print('Correlation with Russian human judgements of Russian word2vec vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(w2v_ru, dataset.similarity)))

Percent of dropped = 3.5%, amount of remanining words = 1348
Percent of dropped = 36.9%, amount of remanining words = 851
Correlation with Russian human judgements of English word2vec vectors: 0.32 (0.00)

Correlation with Russian human judgements of Russian word2vec vectors: 0.37 (0.00)


In [81]:
df_en = ew.copy()

dataset = load_ru_sim_dataset(df_en, rusvectores, True, rusvectores=True)
dataset.to_csv('temp.csv')
dataset = load_sim_dataset(read_csv('temp.csv'), googlenews, True)

w2v_ru = make_ru_sims_dataset(dataset, rusvectores, rusvectores=True)
print('Correlation with English human judgements of Russian word2vec vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(w2v_ru, dataset.similarity)))
print()

w2v_en = make_sims_dataset(dataset, googlenews)
print('Correlation with Enlglish human judgements of English word2vec vectors: {:0.2f} ({:0.2f})'.format(*spearmanr(w2v_en, dataset.similarity)))

Percent of dropped = 84.0%, amount of remanining words = 1830
Percent of dropped = 0.2%, amount of remanining words = 1826
Correlation with English human judgements of Russian word2vec vectors: 0.24 (0.00)

Correlation with Enlglish human judgements of English word2vec vectors: 0.39 (0.00)
